In [1]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
from typing import List
import scipy.stats as sts
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

import util
from pathlib import Path
from class_manipulates_path import ManipulatePath
from class_preprocessing_refactor import Preprocessing
from class_format_data import FormatData
util.init()

current_path = sys.path[0]

In [2]:
def plot_variable(df, column_name, intervalos=None):
    """
    Plota um gráfico de dispersão para uma variável de um DataFrame.

    Parâmetros:
        - df: DataFrame pandas contendo os dados.
        - column_name: Nome da coluna que você deseja plotar.
        - intervalos: Lista de intervalos para plotar linhas verticais.
    """
    plt.figure(figsize=(20, 10))
    plt.scatter(df.index.values, df[column_name].values, color='darkcyan', alpha=0.5)
    plt.xlabel('Índice')
    plt.ylabel(column_name)
    plt.title(f'Gráfico de dispersão para a variável "{column_name}"')


    # Plotar linhas verticais para cada intervalo
    if intervalos:
        for intervalo in intervalos:
            plt.axvline(x=intervalo, linestyle='--', color='red')

    plt.show()

In [3]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [4]:
manipulate_path = ManipulatePath()

preprocessing = Preprocessing()

format_data = FormatData()

path_raw_data = manipulate_path.get_path_raw_data()

abnormal_classes_codes = [1, 2, 5, 6, 7, 8]

In [5]:
# Gets all real instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(path_raw_data, 
                                                       real=True,
                                                       simulated=False, 
                                                       drawn=False),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [6]:
falha = 5
falha_2 = 105

In [7]:
real_instances[real_instances["class_code"] == falha]

,class_code,instance_path
27,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
28,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
29,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
30,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
31,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
32,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
33,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
34,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
35,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...
36,5,c:\Users\marce\Documents\MEGA\0_PUC-Rio\2_Peri...


In [8]:
real_instances[real_instances["class_code"] == falha]["instance_path"].values

array([WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00015_20170620040530.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00016_20180404222255.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00016_20180426102531.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00016_20180426105723.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00016_20180517184536.csv'),
       WindowsPath('c:/Users/marce/Documents/MEGA/0_PUC-Rio/2_Periodo/2_Data_Science/2_Trabalho_Final/3_3W/1_data/1_raw_data/5/WELL-00017_20140314135248.csv'),
       WindowsPath('c:/Users/marce/Docum

In [9]:
file_paths = real_instances[real_instances["class_code"] == falha]["instance_path"].values
df_output = pd.DataFrame()

for file_path in file_paths:
    df_data = pd.read_csv(file_path)

    df_data = FormatData.set_index_dataframe(df_data=df_data,
                                         column_name="timestamp")
    quantidade = ((df_data['class'] == falha) | (df_data['class'] == falha_2)).sum()
    if quantidade != 0:
        print(f"1 se repete {quantidade}")
        print(f"Index min {df_data.index.min()} index max {df_data.index.max()}")

    df_output = pd.concat([df_output, df_data], axis=0)

1 se repete 8959
Index min 2017-06-20 04:05:30 index max 2017-06-20 19:23:52
1 se repete 2114
Index min 2018-04-04 22:22:55 index max 2018-04-05 02:58:55
1 se repete 1104
Index min 2018-04-26 10:25:31 index max 2018-04-26 14:44:04
1 se repete 2796
Index min 2018-04-26 10:57:23 index max 2018-04-26 15:44:20
1 se repete 10461
Index min 2018-05-17 18:45:36 index max 2018-05-18 01:42:55
1 se repete 138359
Index min 2014-03-14 13:52:48 index max 2014-03-16 08:33:14
1 se repete 32072
Index min 2014-03-17 12:34:19 index max 2014-03-18 01:30:00
1 se repete 39357
Index min 2014-03-17 22:59:27 index max 2014-03-18 13:57:58
1 se repete 31870
Index min 2014-03-18 13:06:03 index max 2014-03-19 02:00:00
1 se repete 17742
Index min 2014-03-19 02:01:30 index max 2014-03-19 11:00:00
1 se repete 45762
Index min 2014-03-19 03:16:16 index max 2014-03-20 04:20:38


In [10]:
df_output

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,
2017-06-20 04:05:30,25008780.0,2.941990e+09,51.93984,5065443.0,33.93840,17542530.0,NaN,0.979444,0.0
2017-06-20 04:05:31,25008810.0,2.941990e+09,51.94104,5068246.0,33.94290,17542890.0,NaN,0.979297,0.0
2017-06-20 04:05:32,25008850.0,2.941990e+09,51.94223,5071049.0,33.94739,17543260.0,NaN,0.979149,0.0
2017-06-20 04:05:33,25008880.0,2.941990e+09,51.94343,5073852.0,33.95188,17543620.0,NaN,0.979001,0.0
2017-06-20 04:05:34,25008920.0,2.941990e+09,51.94463,5076655.0,33.95638,17543980.0,NaN,0.978853,0.0
...,...,...,...,...,...,...,...,...,...
2014-03-20 04:20:34,32308910.0,4.631294e+06,87.30427,1311047.0,31.23388,6356541.0,NaN,0.000000,5.0
2014-03-20 04:20:35,32308910.0,4.630961e+06,87.30622,1310748.0,31.23269,6356974.0,NaN,0.000000,5.0
2014-03-20 04:20:36,32308910.0,4.630629e+06,87.30817,1310449.0,31.23150,6357407.0,NaN,0.000000,5.0


In [11]:
df_output = df_output[~df_output.index.duplicated()]

In [12]:
df_output.shape

(498952, 9)

In [13]:
df_aux = df_output.drop(columns=["QGL", "T-JUS-CKGL"])
df_aux

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,class
timestamp,,,,,,,
2017-06-20 04:05:30,25008780.0,2.941990e+09,51.93984,5065443.0,33.93840,17542530.0,0.0
2017-06-20 04:05:31,25008810.0,2.941990e+09,51.94104,5068246.0,33.94290,17542890.0,0.0
2017-06-20 04:05:32,25008850.0,2.941990e+09,51.94223,5071049.0,33.94739,17543260.0,0.0
2017-06-20 04:05:33,25008880.0,2.941990e+09,51.94343,5073852.0,33.95188,17543620.0,0.0
2017-06-20 04:05:34,25008920.0,2.941990e+09,51.94463,5076655.0,33.95638,17543980.0,0.0
...,...,...,...,...,...,...,...
2014-03-20 04:20:34,32308910.0,4.631294e+06,87.30427,1311047.0,31.23388,6356541.0,5.0
2014-03-20 04:20:35,32308910.0,4.630961e+06,87.30622,1310748.0,31.23269,6356974.0,5.0
2014-03-20 04:20:36,32308910.0,4.630629e+06,87.30817,1310449.0,31.23150,6357407.0,5.0


In [14]:
for column_name in df_aux.columns:
    percent = Preprocessing.check_null_dataframe(df_data=df_output, column_name=column_name)
    print(f"{column_name} null data is {percent}")

P-PDG null data is 0.32488095047218973
P-TPT null data is 0.32488095047218973
T-TPT null data is 0.3240792701502349
P-MON-CKP null data is 0.32568263079414456
T-JUS-CKP null data is 0.32568263079414456
P-JUS-CKGL null data is 0.32568263079414456
class null data is 13.67887091343456


In [15]:
df_aux = df_aux.dropna()
df_aux.shape

(429739, 7)

In [16]:
(df_aux['class'] == 105).sum() / df_aux.shape[0]

0.7389717945078291

# KNN

In [17]:
contagem_classes = df_aux['class'].value_counts()

# Calcular a porcentagem de cada classe
porcentagem_classes = contagem_classes / len(df_aux) * 100
porcentagem_classes

class
105.0    73.897179
0.0      23.294372
5.0       2.808449
Name: count, dtype: float64

In [18]:
# Separar as features (X) e os rótulos (y)
X = df_aux.drop('class', axis=1)
y = df_aux['class']

In [19]:
X

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL
timestamp,,,,,,
2017-06-20 04:05:30,25008780.0,2.941990e+09,51.93984,5065443.0,33.93840,17542530.0
2017-06-20 04:05:31,25008810.0,2.941990e+09,51.94104,5068246.0,33.94290,17542890.0
2017-06-20 04:05:32,25008850.0,2.941990e+09,51.94223,5071049.0,33.94739,17543260.0
2017-06-20 04:05:33,25008880.0,2.941990e+09,51.94343,5073852.0,33.95188,17543620.0
2017-06-20 04:05:34,25008920.0,2.941990e+09,51.94463,5076655.0,33.95638,17543980.0
...,...,...,...,...,...,...
2014-03-20 04:20:34,32308910.0,4.631294e+06,87.30427,1311047.0,31.23388,6356541.0
2014-03-20 04:20:35,32308910.0,4.630961e+06,87.30622,1310748.0,31.23269,6356974.0
2014-03-20 04:20:36,32308910.0,4.630629e+06,87.30817,1310449.0,31.23150,6357407.0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [22]:
modelo = KNeighborsClassifier(n_neighbors=5)

In [23]:
modelo.fit(X_train, y_train)

KNeighborsClassifier()

In [24]:
y_pred = modelo.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

0.9741549153751881

In [26]:
precision = precision_score(y_test,  y_pred, average=None)
recall = recall_score(y_test,  y_pred, average=None)
f1 = f1_score(y_test,  y_pred, average=None)

for i in range(len(precision)):
    print(f'Classe {porcentagem_classes.index.values[i]}:')
    print(f'Precision: {precision[i]:.3f}')
    print(f'Recall: {recall[i]:.3f}')
    print(f'F1-score: {f1[i]:.3f}\n')

Classe 105.0:
Precision: 0.944
Recall: 0.947
F1-score: 0.945

Classe 0.0:
Precision: 0.989
Recall: 0.997
F1-score: 0.993

Classe 5.0:
Precision: 0.983
Recall: 0.982
F1-score: 0.983



In [27]:
confusion_matrix(y_test.values, y_pred)

array([[28444,     0,  1587],
       [    0,  3610,    11],
       [ 1693,    41, 93536]], dtype=int64)

# MLP

In [27]:
modelo = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [28]:
modelo.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

In [29]:
y_pred = modelo.predict(X_test)

In [30]:
accuracy_score(y_test, y_pred)

0.9607747320085013